# Catboost


In [1]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 5.2 MB/s eta 0:00:00


In [2]:
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split
import pandas as pd

df_all = pd.read_csv('combined_ema.csv')

columns_to_drop = [
    'stressor_partner', 'stressor_fam', 'stressor_breakdown',
    'stressor_money', 'stressor_selfcare', 'stressor_health',
    'stressor_otherhealth', 'stressor_household', 'stressor_child',
    'stressor_discrimination', 'stressor_none', 'moststressful',
    'moststressful_time', 'work_location', 'attend_fidam', 'attend_fidpm',
    'attend_hasp', 'attend_pgy1did', 'attend_pgy2did', 'attend_pgy3did',
    'attend_none', 'work_start', 'work_end', 'jobperformance_best',
    'jobsatisfaction', "jobperformance2","date","Sleep1BeginTimestamp",
    'Sleep1EndTimestamp', 'Sleep2BeginTimestamp', 'Sleep2EndTimestamp', 'Sleep3BeginTimestamp', 'Sleep3EndTimestamp'
]

df_all.drop(columns=columns_to_drop, inplace=True, errors='ignore')
aud = [f"audio_{i}" for i in range(87)]

cols_to_select = ['jobperformance']
cols_to_select.extend(aud)
df_all = df_all.loc[:,cols_to_select]

import numpy as np

df_all['jobperformance'] = df_all['jobperformance'].replace(' ', np.nan)

df_all.dropna(subset=['jobperformance'], inplace=True)

df_all['jobperformance'] = df_all['jobperformance'].astype(float)

Y = df_all['jobperformance'].astype(float)  # Ensure it is numeric
X = df_all.drop(['jobperformance'], axis=1)

for col in X.select_dtypes(include=['object']).columns:
    X[col] = X[col].astype('category')

X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=42)
for col in X_train.select_dtypes(include=['category']).columns:
    X_train[col] = X_train[col].astype(str).fillna('missing')
    X_val[col] = X_val[col].astype(str).fillna('missing')

model = CatBoostRegressor(verbose=False)
model.fit(
    X_train, Y_train,
    cat_features=[X_train.columns.get_loc(c) for c in X_train.select_dtypes(['object']).columns]  # Now it expects object type since all are converted to string
)

predictions = model.predict(X_val)



In [3]:
from sklearn.metrics import mean_squared_error, r2_score


import numpy as np

def concordance_correlation_coefficient(y_true, y_pred):
    """
    Compute the Concordance Correlation Coefficient for two arrays.
    """
    mean_true = np.mean(y_true)
    mean_pred = np.mean(y_pred)

    var_true = np.var(y_true)
    var_pred = np.var(y_pred)
    covariance = np.mean((y_pred - mean_pred) * (y_true - mean_true))

    ccc = (2 * covariance) / (var_pred + var_true + (mean_pred - mean_true) ** 2)

    return ccc


train_predictions = model.predict(X_train)

val_predictions = model.predict(X_val)

train_mse = mean_squared_error(Y_train, train_predictions)
train_r2 = r2_score(Y_train, train_predictions)
print("Training Data - Mean Squared Error: {:.2f}".format(train_mse))
print("Training Data - R^2 Score: {:.2f}".format(train_r2))

val_mse = mean_squared_error(Y_val, val_predictions)
val_r2 = r2_score(Y_val, val_predictions)
print("Validation Data - Mean Squared Error: {:.2f}".format(val_mse))
print("Validation Data - R^2 Score: {:.2f}".format(val_r2))



train_ccc = concordance_correlation_coefficient(Y_train, train_predictions)
val_ccc = concordance_correlation_coefficient(Y_val, val_predictions)

print("Training Data - CCC: {:.2f}".format(train_ccc))
print("Validation Data - CCC: {:.2f}".format(val_ccc))


Training Data - Mean Squared Error: 0.66
Training Data - R^2 Score: 0.48
Validation Data - Mean Squared Error: 1.21
Validation Data - R^2 Score: 0.15
Training Data - CCC: 0.65
Validation Data - CCC: 0.25


In [4]:
X_val.keys()

Index(['audio_0', 'audio_1', 'audio_2', 'audio_3', 'audio_4', 'audio_5',
       'audio_6', 'audio_7', 'audio_8', 'audio_9', 'audio_10', 'audio_11',
       'audio_12', 'audio_13', 'audio_14', 'audio_15', 'audio_16', 'audio_17',
       'audio_18', 'audio_19', 'audio_20', 'audio_21', 'audio_22', 'audio_23',
       'audio_24', 'audio_25', 'audio_26', 'audio_27', 'audio_28', 'audio_29',
       'audio_30', 'audio_31', 'audio_32', 'audio_33', 'audio_34', 'audio_35',
       'audio_36', 'audio_37', 'audio_38', 'audio_39', 'audio_40', 'audio_41',
       'audio_42', 'audio_43', 'audio_44', 'audio_45', 'audio_46', 'audio_47',
       'audio_48', 'audio_49', 'audio_50', 'audio_51', 'audio_52', 'audio_53',
       'audio_54', 'audio_55', 'audio_56', 'audio_57', 'audio_58', 'audio_59',
       'audio_60', 'audio_61', 'audio_62', 'audio_63', 'audio_64', 'audio_65',
       'audio_66', 'audio_67', 'audio_68', 'audio_69', 'audio_70', 'audio_71',
       'audio_72', 'audio_73', 'audio_74', 'audio_75', 'audio_

# XGBoost

In [5]:
X_train_xg = X_train.copy()
X_val_xg = X_val.copy()
Y_train_xg = Y_train.copy()
Y_val_xg = Y_val.copy()

In [6]:
X_train_xg = X_train_xg.astype('category')
X_val_xg = X_val.astype('category')

In [7]:
from sklearn.preprocessing import LabelEncoder

label_enc = LabelEncoder()
for column in X_train_xg.columns:
    if X_train_xg[column].dtype == 'object':
        X_train_xg[column].fillna('missing', inplace=True)
        X_val_xg[column].fillna('missing', inplace=True)

        X_train_xg[column] = label_enc.fit_transform(X_train_xg[column])
        X_val_xg[column] = label_enc.transform(X_val_xg[column])


In [8]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

xgb_model = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.6, learning_rate = 0.1,
                             max_depth = 5, alpha = 10, n_estimators = 100,
                                 enable_categorical=True,)

xgb_model.fit(X_train_xg, Y_train_xg)

xgb_train_predictions = xgb_model.predict(X_train_xg)

xgb_val_predictions = xgb_model.predict(X_val_xg)

xgb_train_mse = mean_squared_error(Y_train_xg, xgb_train_predictions)
xgb_train_r2 = r2_score(Y_train_xg, xgb_train_predictions)

xgb_val_mse = mean_squared_error(Y_val_xg, xgb_val_predictions)
xgb_val_r2 = r2_score(Y_val_xg, xgb_val_predictions)

print("XGBoost Training Data - Mean Squared Error: {:.2f}".format(xgb_train_mse))
print("XGBoost Training Data - R² Score: {:.2f}".format(xgb_train_r2))
print("XGBoost Validation Data - Mean Squared Error: {:.2f}".format(xgb_val_mse))
print("XGBoost Validation Data - R² Score: {:.2f}".format(xgb_val_r2))

def concordance_correlation_coefficient(y_true, y_pred):
    mean_true = np.mean(y_true)
    mean_pred = np.mean(y_pred)
    var_true = np.var(y_true)
    var_pred = np.var(y_pred)
    covariance = np.mean((y_pred - mean_pred) * (y_true - mean_true))
    ccc = (2 * covariance) / (var_pred + var_true + (mean_pred - mean_true) ** 2)
    return ccc

xgb_train_ccc = concordance_correlation_coefficient(Y_train_xg, xgb_train_predictions)
xgb_val_ccc = concordance_correlation_coefficient(Y_val_xg, xgb_val_predictions)

print("XGBoost Training Data - CCC: {:.2f}".format(xgb_train_ccc))
print("XGBoost Validation Data - CCC: {:.2f}".format(xgb_val_ccc))


XGBoost Training Data - Mean Squared Error: 0.98
XGBoost Training Data - R² Score: 0.23
XGBoost Validation Data - Mean Squared Error: 1.40
XGBoost Validation Data - R² Score: 0.02
XGBoost Training Data - CCC: 0.30
XGBoost Validation Data - CCC: 0.04


# LightGBM

In [9]:
Xt_backup = X_train.copy()
Xv_backup = X_val.copy()
Yt_backup = Y_train.copy()
Yv_backup = Y_val.copy()

In [10]:
X_train_l = X_train.copy()
X_val_l = X_val.copy()
Y_train_l = Y_train.copy()
Y_val_l = Y_val.copy()

In [13]:
categorical_features = []
for column in X_train.columns:
    if X_train[column].dtype == 'object':
        X_train[column] = X_train[column].astype('category')
        X_val[column] = X_val[column].astype('category')
        categorical_features.append(column)


In [14]:
import lightgbm as lgb
from sklearn.metrics import mean_squared_error, r2_score

# Initialize LightGBM
lgb_model = lgb.LGBMRegressor(
    objective='regression',
    num_leaves=31,
    learning_rate=0.05,
    n_estimators=100,
    categorical_feature=categorical_features  # Ensure categorical features are specified
)

# Fit the model
lgb_model.fit(
    X_train, Y_train,
    eval_set=[(X_val, Y_val)],
    eval_metric='mse',
)

# Predictions
lgb_train_predictions = lgb_model.predict(X_train)
lgb_val_predictions = lgb_model.predict(X_val)



/usr/local/lib/python3.10/dist-packages/lightgbm/basic.py:1873: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning(f'{key} keyword has been found in `params` and will be ignored.\n'
/usr/local/lib/python3.10/dist-packages/lightgbm/basic.py:1873: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning(f'{key} keyword has been found in `params` and will be ignored.\n'


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001048 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 7448
[LightGBM] [Info] Number of data points in the train set: 542, number of used features: 87
[LightGBM] [Info] Start training from score 2.966790
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, b

In [15]:
# Metrics
print("LightGBM")
train_mse = mean_squared_error(Y_train, lgb_train_predictions)
train_r2 = r2_score(Y_train, lgb_train_predictions)
val_mse = mean_squared_error(Y_val, lgb_val_predictions)
val_r2 = r2_score(Y_val, lgb_val_predictions)

print("Training MSE: {:.3f}, R2: {:.2f}".format(train_mse, train_r2))
print("Validation MSE: {:.3f}, R2: {:.2f}".format(val_mse, val_r2))

train_ccc = concordance_correlation_coefficient(Y_train, lgb_train_predictions)
val_ccc = concordance_correlation_coefficient(Y_val_l, lgb_val_predictions)

print("Training CCC: {:.3f}".format(train_ccc))
print("Validation CCC: {:.3f}".format(val_ccc))


LightGBM
Training MSE: 0.752, R2: 0.41
Validation MSE: 1.195, R2: 0.16
Training CCC: 0.534
Validation CCC: 0.256


# Sk-learn

In [16]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

categorical_features = X_train.select_dtypes(include=['object', 'category']).columns.tolist()

numeric_features = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()
numeric_transformer = SimpleImputer(strategy='median')

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

X_train_prepared = preprocessor.fit_transform(X_train)
X_val_prepared = preprocessor.transform(X_val)


In [17]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Lasso

# Random Forest
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train_prepared, Y_train)
rf_train_preds = rf_model.predict(X_train_prepared)
rf_val_preds = rf_model.predict(X_val_prepared)

# Support Vector Machine
svm_model = SVR()
svm_model.fit(X_train_prepared, Y_train)
svm_train_preds = svm_model.predict(X_train_prepared)
svm_val_preds = svm_model.predict(X_val_prepared)

# Linear Regression
lr_model = LinearRegression()
lr_model.fit(X_train_prepared, Y_train)
lr_train_preds = lr_model.predict(X_train_prepared)
lr_val_preds = lr_model.predict(X_val_prepared)

# Lasso Regression
lasso_model = Lasso(alpha=0.1)
lasso_model.fit(X_train_prepared, Y_train)
lasso_train_preds = lasso_model.predict(X_train_prepared)
lasso_val_preds = lasso_model.predict(X_val_prepared)

# Evaluation
from sklearn.metrics import mean_squared_error, r2_score

def evaluate_model(name, train_actuals, train_preds, val_actuals, val_preds):
    train_mse = mean_squared_error(train_actuals, train_preds)
    train_r2 = r2_score(train_actuals, train_preds)
    val_mse = mean_squared_error(val_actuals, val_preds)
    val_r2 = r2_score(val_actuals, val_preds)
    print(f"{name} - Training MSE: {train_mse:.3f}, R²: {train_r2:.2f}")
    print(f"{name} - Validation MSE: {val_mse:.3f}, R²: {val_r2:.2f}")

evaluate_model('Random Forest', Y_train, rf_train_preds, Y_val, rf_val_preds)
train_ccc = concordance_correlation_coefficient(Y_train, rf_train_preds)
val_ccc = concordance_correlation_coefficient(Y_val, rf_val_preds)

print("Training CCC: {:.3f}".format(train_ccc))
print("Validation CCC: {:.3f}".format(val_ccc))

evaluate_model('SVM', Y_train, svm_train_preds, Y_val, svm_val_preds)
train_ccc = concordance_correlation_coefficient(Y_train, svm_train_preds)
val_ccc = concordance_correlation_coefficient(Y_val, svm_val_preds)

print("Training CCC: {:.3f}".format(train_ccc))
print("Validation CCC: {:.3f}".format(val_ccc))
evaluate_model('Linear Regression', Y_train, lr_train_preds, Y_val, lr_val_preds)
train_ccc = concordance_correlation_coefficient(Y_train, lr_train_preds)
val_ccc = concordance_correlation_coefficient(Y_val, lr_val_preds)

print("Training CCC: {:.3f}".format(train_ccc))
print("Validation CCC: {:.3f}".format(val_ccc))
evaluate_model('Lasso', Y_train, lasso_train_preds, Y_val, lasso_val_preds)
train_ccc = concordance_correlation_coefficient(Y_train, lasso_train_preds)
val_ccc = concordance_correlation_coefficient(Y_val, lasso_val_preds)

print("Training CCC: {:.3f}".format(train_ccc))
print("Validation CCC: {:.3f}".format(val_ccc))




Random Forest - Training MSE: 0.742, R²: 0.42
Random Forest - Validation MSE: 1.235, R²: 0.13
Training CCC: 0.532
Validation CCC: 0.202
SVM - Training MSE: 1.285, R²: -0.00
SVM - Validation MSE: 1.381, R²: 0.03
Training CCC: 0.009
Validation CCC: 0.038
Linear Regression - Training MSE: 0.997, R²: 0.22
Linear Regression - Validation MSE: 700594.983, R²: -491752.82
Training CCC: 0.364
Validation CCC: 0.000
Lasso - Training MSE: 1.135, R²: 0.12
Lasso - Validation MSE: 1.307, R²: 0.08
Training CCC: 0.193
Validation CCC: 0.169


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.026e+02, tolerance: 6.954e-02
  model = cd_fast.enet_coordinate_descent(
